# QUBO Portfolio Optimisation for a Single Football Match

We model 5 related bets on one match as binary variables:

1. `HW`   – Home Win  
2. `O25`  – Over 2.5 goals  
3. `BTTS` – Both Teams To Score: Yes  
4. `CS10` – Correct Score 1–0  
5. `H_1`  – Home -1 handicap  

We:
- compute linear QUBO coefficients `w_i` from odds + our own probabilities,  
- compute quadratic coefficients `Q_ij` from a (toy) correlation matrix,  
- build the full QUBO, check all 2^5 portfolios classically,  

In [1]:
import numpy as np
import pandas as pd
import itertools

import orbit 

def expected_value(p, odds):
    """
    Expected profit per unit stake:
    EV = p * odds - 1
    """
    return p * odds - 1.0

def portfolio_energy_qubo(x, w, Q):
    """
    QUBO energy:
    E(x) = sum_i w_i x_i + sum_{i<j} Q_ij x_i x_j
    x : 1D numpy array of 0/1 of length N
    """
    linear = np.dot(w, x)
    quadratic = 0.0
    N = len(x)
    for i in range(N):
        for j in range(i+1, N):
            quadratic += Q[i, j] * x[i] * x[j]
    return linear + quadratic

In [2]:
# 5 related markets / bets on a single match
bets = ["HW", "O25", "BTTS_Y", "CS_1_0", "H_minus_1"]

# Example live odds (decimal)
odds = np.array([1.90, 1.95, 2.10, 7.50, 2.60])

# Your model's true probabilities
p_true = np.array([0.55, 0.54, 0.50, 0.12, 0.42])

# Expected value per bet
EV = expected_value(p_true, odds)

# Linear QUBO coefficients: w_i = -EV_i  (no liquidity penalty yet)
w = -EV

linear_df = pd.DataFrame({
    "bet": bets,
    "odds": odds,
    "p_true": p_true,
    "EV": EV,
    "w_i": w
})
linear_df

,bet,odds,p_true,EV,w_i
0,HW,1.90,0.55,0.045,-0.045
1,O25,1.95,0.54,0.053,-0.053
2,BTTS_Y,2.10,0.50,0.050,-0.050
3,CS_1_0,7.50,0.12,-0.100,0.100
4,H_minus_1,2.60,0.42,0.092,-0.092


In [11]:
# Toy payoff/return correlation matrix between these 5 bets
rho = np.array([
    [ 1.0,  0.6,  0.4, -0.3,  0.7],   # HW
    [ 0.6,  1.0,  0.7, -0.5,  0.4],   # O25
    [ 0.4,  0.7,  1.0, -0.4,  0.3],   # BTTS_Y
    [-0.3, -0.5, -0.4,  1.0,  0.1],   # CS_1_0
    [ 0.7,  0.4,  0.3,  0.1,  1.0],   # H_minus_1
])

# Risk-penalty scaling
lambda_corr = 1

# Off-diagonal QUBO couplings from correlations
Q = lambda_corr * rho
np.fill_diagonal(Q, 0.0)   # we keep linear effects in w only

Q_df = pd.DataFrame(Q, index=bets, columns=bets)
Q_df

,HW,O25,BTTS_Y,CS_1_0,H_minus_1
HW,0.0,0.6,0.4,-0.3,0.7
O25,0.6,0.0,0.7,-0.5,0.4
BTTS_Y,0.4,0.7,0.0,-0.4,0.3
CS_1_0,-0.3,-0.5,-0.4,0.0,0.1
H_minus_1,0.7,0.4,0.3,0.1,0.0


In [12]:
N = len(bets)
all_results = []

for bits in itertools.product([0, 1], repeat=N):
    x = np.array(bits)
    E = -1*portfolio_energy_qubo(x, w, Q)
    all_results.append({"x": bits, "E": E, "num_bets": x.sum()})

results_df = pd.DataFrame(all_results).sort_values("E").reset_index(drop=True)
results_df.head(32)

,x,E,num_bets
0,"(1, 1, 1, 0, 1)",-2.860,4
1,"(1, 1, 1, 1, 1)",-1.860,5
2,"(1, 1, 1, 0, 0)",-1.552,3
3,"(1, 1, 0, 0, 1)",-1.510,3
4,"(1, 0, 1, 0, 1)",-1.213,3
5,"(0, 1, 1, 0, 1)",-1.205,3
6,"(1, 1, 0, 1, 1)",-0.910,4
7,"(1, 0, 1, 1, 1)",-0.713,4
8,"(0, 1, 1, 0, 0)",-0.597,2
9,"(1, 0, 0, 0, 1)",-0.563,2


In [13]:
N = len(bets)

# J_ij = Q_ij / 4   (symmetric, zero diagonal)
J = Q /4

# h_i = w_i/2 + (1/4) * sum_{j != i} Q_ij
h = np.zeros(N)
for i in range(N):
    h[i] = w[i]/2 + 0.25*np.sum(Q[i, :])  # Q[i,i] is 0, so it's fine

ising_J = J.copy()
ising_h = h.copy()

print("h (local fields):", ising_h)
print("\nJ (couplings):")
pd.DataFrame(ising_J, index=bets, columns=bets)

h (local fields): [ 0.3275  0.2735  0.225  -0.225   0.329 ]

J (couplings):


,HW,O25,BTTS_Y,CS_1_0,H_minus_1
HW,0.000,0.150,0.100,-0.075,0.175
O25,0.150,0.000,0.175,-0.125,0.100
BTTS_Y,0.100,0.175,0.000,-0.100,0.075
CS_1_0,-0.075,-0.125,-0.100,0.000,0.025
H_minus_1,0.175,0.100,0.075,0.025,0.000


In [22]:
result = orbit.optimize_ising(
    -1*ising_J,
    -1*ising_h,
    n_replicas=1,
    full_sweeps=5_000,   # you can increase later
    beta_initial=0.35,
    beta_end=3.5,
    beta_step_interval=1,
)

print("Minimum Ising energy (ORBIT):", result.min_cost)
print("Minimum spin state (ORBIT):   ", result.min_state)

[2025-12-03 16:43:37] INFO - orbit.simulator: Simulation starting...
[2025-12-03 16:43:38] INFO - orbit.simulator: Simulation completed in 1.01 seconds
Minimum Ising energy (ORBIT): -3.48
Minimum spin state (ORBIT):    [1 1 1 0 1]


In [18]:
s_star = np.array(result.min_state)      # spins in {-1, +1}
x_star = (1 + s_star) // 2               # convert to {0,1}

chosen_bets = [b for b, bit in zip(bets, x_star) if bit == 1]

print("Optimal bitstring x* (from ORBIT):", x_star.tolist())
print("Selected bets:", chosen_bets)

# Compare ORBIT energy in QUBO space to classical check
E_orbit_qubo = portfolio_energy_qubo(x_star, w, Q)
print("QUBO energy of ORBIT solution:", E_orbit_qubo)

Optimal bitstring x* (from ORBIT): [1, 1, 1, 0, 1]
Selected bets: ['HW', 'O25', 'BTTS_Y', 'H_minus_1']
QUBO energy of ORBIT solution: 2.8599999999999994


In [20]:
import math
import random

def classical_sa_ising(h, J, sweeps=50_000, beta_initial=0.35, beta_end=3.5, beta_steps=10, seed=None):
    """
    Classical simulated annealing for the Ising energy:
        E(s) = sum_i h_i s_i + sum_{i<j} J_ij s_i s_j
    s_i in {-1, +1}

    Returns:
        best_state (np.array of spins),
        best_energy (float)
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    N = len(h)

    # Initialise with random spins in {-1, +1}
    s = np.random.choice([-1, 1], size=N)

    # Helper to compute energy
    def energy(spins):
        # E = h.s + sum_{i<j} J_ij s_i s_j
        e = np.dot(h, spins)
        for i in range(N):
            for j in range(i+1, N):
                e += J[i, j] * spins[i] * spins[j]
        return e

    E = energy(s)
    best_s = s.copy()
    best_E = E

    # Schedule over beta (inverse temperature)
    betas = np.linspace(beta_initial, beta_end, beta_steps)

    for beta in betas:
        for _ in range(sweeps // beta_steps):
            # pick random spin
            i = np.random.randint(0, N)
            s_new = s.copy()
            s_new[i] *= -1  # flip spin i

            # compute energy difference ΔE
            # to be efficient, compute local contribution only
            dE = 2 * h[i] * s[i]
            for j in range(N):
                if j == i:
                    continue
                dE += 2 * J[min(i, j), max(i, j)] * s[i] * s[j]

            if dE <= 0 or random.random() < math.exp(-beta * dE):
                s = s_new
                E += dE
                if E < best_E:
                    best_E = E
                    best_s = s.copy()

    return best_s, best_E

Pull the API - save static data (odds and implied probabilities) for a small, medium and big game
Correlation Calculations - the frequency of getting a correlation (2weeks ahead of time of match) 

Benchmarking against the classical solutions (market size) 
Application to other sectors -> finance (asset groups)

Constraints on the resource -> moving away from the trivial solution and adding features to the expectation values
Optimising the edges -> sparse correlation matrix 

low priority options pricing 